# WordPress 서버 클러스터 구성

## 3 tier architecture
- 정형화된 3개의 물리적/논리적 티어로 구성된 어플리케이션 아키텍쳐
- 일반적인 티어
    - Presentation tier : 사용자와 직접 소통하는 티어. 주로 HTML, CSS와 같은 유저를 위한 UI로 구성
    - Application tier : 실제 로직을 처리하는 티어로 다양한 프로그래밍 언어(PHP, JAVA, Python) 등으로 구성
    - Data tier : 데이터를 저장하는 티어. 주로 데이터베이스 혹은 데이터를 처리하는 저장 공간으로 구성
- AWS에서는 ALB / Web EC2 / RDS 3가지 티어로 구성 예정

### 데모 - 3티어 아키텍쳐를 활용하여 워드프레스 서버를 위한 웹 서버 클러스터 구성
- EC2 Autoscaling, ELB로 고가용성 확보
- EFS를 활용해서 공유 스토리지에 워드프레스 설치

### 데모 순서
- VPC 생성, 퍼블릭/프라이빗 서브넷 각 2개
- RDS 서브넷 그룹 생성, RDS 프로비전
- EFS 생성
- S3 버킷 생성 : wp-config 업데이트 및 업로드
- 유저데이터 준비 및 EC2 생성 : 해당 EC2로 워드프레스 생성, AMI 생성
- 해당 AMI로 런치 템플릿 생성. Autoscaling Group + ALB
- Cors/보안그룹 때문에 최초 설치 경로를 바라보는 파일 로딩 불가능
    - 직접 로그인하여 워드프레스의 주소 및 경로 수정이 필요

### 실습
1. IAM 역할 생성 : EC2가 S3에서 파일을 가져올 수 있도록
    - S3fullaccess 권한 부여
2. VPC 생성
    - VPC 엔드포인트 없음 선택
3. RDS 생성
    - 서브넷 그룹 생성
        - 방금 생성한 VPC 선택
        - VPC의 서브넷과 동일한 가용 영역 선택
        - 프라이빗 서브넷 선택
    - 데이터베이스 생성
        - MySQL 선택
        - 프리티어
        - 마스터암호 입력 : abcd1234
        - VPC 선택
        - 추가 구성 : 초기 데이터베이스 이름 wordpress
4. VPC 보안그룹 수정
    - 새로 생성된 보안그룹 수정 : 인바운드 규칙 모두 허용
5. EFS 생성
    - VPC 선택
6. S3 버킷 생성
7. 소스 파일 수정
    - wp-config 수정
        - DB Host Name : RDS 엔드포인트
    - demo-3tier 수정
        - EFS ID : EFS의 이름 옆에 있는 아이디를 넣자
        - S3 버킷 : 버킷 이름 넣자
8. 소스 파일 업로드
    - S3 버킷에 wp-config.php 업로드
9. EC2 생성
    - 키페어 사용
    - VPC, 퍼블릭 서브넷 선택
    - 퍼블릭 아이피 활성화
    - 디폴트 보안그룹 선택
    - 고급 세부 정보 : IAM 인스턴스 프로파일 선택
    - 유저데이터 스크립트 넣기
10. EC2 연결
    - sudo -s
    - cd /var/www/html : wordpress 나와야함
    - 퍼블릭 아이피+/wordpress로 접속해보자 : 언어 선택 창이 보여야함
    - username : admin // password : abcd1234 //  confirm password // 이메일 입력, 워드프레스 설치
    - public IP + /wordpress 접속해보자
11. 이미지 생성, 오토스케일에 넣기
    - 이미지 생성
    - 시작 템플릿 생성
        - 방금 만든 AMI 선택
        - 인스턴스 유형은 t2.micro
    - 오토스케일 그룹 생성
        - 시작 템플릿 적용
        - VPC 수정, 가용영역 모두 선택
        - 대상 그룹 생성
            - VPC 선택
            - 상태 검사 경로 : /wordpress
            - 성공 코드 : 301
        - 로드 밸런서 대상 그룹으로 지정
        - ELB 상태 확인 켜기
        - 원하는 용량 2, 최소 0, 최대 2
        - 태그 : Name // demo-3tier-wordpress
12. 로드 밸런서 생성
    - VPC 선택, 가용영역 둘 다 선택 (퍼블릭 서브넷)
    - 대상 그룹 선택
    - 로드 밸런서의 DNS 이름 + /wordpress 를 주소창에 입력해보자
        - 검사 : cors 에러가 뜬 것을 확인
        - dns이름+/wordpress/wp-admin을 주소창에 입력, 로그인
        - 세팅 : wordpress 주소를 로드 밸런서의 DNS 이름+/wordpress으로 바꿔주자
13. 인스턴스로 직접 접근할 수 없도록 보안 그룹을 수정해보자
    - EC2 보안 그룹 생성(ALB용)
        - VPC 선택
        - 들어오는 트래픽 http만 허용, 소스는 모두
        - 알아볼 수 있도록 보안그룹 이름 바꿔주고
    - 보안그룹 하나 더 생성(EC2용)
        - VPC 선택
        - 모든 트래픽 허용, 소스는 ALB 보안그룹
        - SSH는 모든 소스 허용
    - 보안 그룹 변경(인스턴스)
    - 시작 템플릿 업데이트 : 새 보안그룹 적용 (EC2용)
    - ASG 업데이트 : 시작 템플릿 최신 버전 적용
    - 로드 밸런서의 보안 그룹 수정
14. 리소스 정리 : ASG, 로드밸런서, RDS 삭제, S3 버킷 삭제, 인스턴스 삭제, EFS 삭제, AMI 등록취소, 스냅샷 삭제

